# Evolver Loop 7 Analysis

## Situation Assessment

After 7 experiments, we're stuck at 70.615102 with a gap of 1.733 points to the target (68.881647).

### What's Been Tried (ALL FAILED):
1. **Fractional translation** - No improvement
2. **Backward propagation** - No improvement
3. **Per-N ensemble from snapshots** - Invalid solutions (overlaps)
4. **SA from scratch** - Worse than baseline
5. **Asymmetric perturbations** - No improvement
6. **External source ensemble** - No valid improvements
7. **Global rotation optimization** - No improvement

### Key Insight from Eazy Optimizer Kernel:
The top kernels use C++ with:
- Complex number orbital moves
- Square pressure gradient descent
- Elastic pulse (periodic squeeze/relax)
- Multi-scale optimization (1e-3 to 1e-9)

### The Problem:
Our Python implementations are too slow and our baseline is already at a strong local optimum created by sophisticated C++ optimizers.

In [1]:
# Let's analyze the baseline to understand where improvements might be possible
import pandas as pd
import numpy as np
from collections import defaultdict

# Load baseline
baseline_path = '/home/code/experiments/002_valid_baseline/submission.csv'
df = pd.read_csv(baseline_path)

configs = defaultdict(list)
for _, row in df.iterrows():
    n = int(row['id'].split('_')[0])
    x = float(str(row['x']).replace('s', ''))
    y = float(str(row['y']).replace('s', ''))
    deg = float(str(row['deg']).replace('s', ''))
    configs[n].append([x, y, deg])

print(f"Loaded {len(configs)} N values")

Loaded 200 N values


In [2]:
# Calculate per-N scores and identify where the most potential improvement lies
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, deg):
    poly = Polygon(zip(TX, TY))
    rotated = affinity.rotate(poly, deg, origin=(0, 0))
    return affinity.translate(rotated, x, y)

def calculate_score(trees, n):
    if not trees:
        return float('inf')
    polys = [create_tree_polygon(t[0], t[1], t[2]) for t in trees]
    bounds = unary_union(polys).bounds
    side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
    return side ** 2 / n

# Calculate all per-N scores
per_n_scores = {}
for n in range(1, 201):
    if n in configs:
        per_n_scores[n] = calculate_score(configs[n], n)

print(f"Total score: {sum(per_n_scores.values()):.6f}")
print(f"\nScore breakdown by range:")
for start, end in [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]:
    range_score = sum(per_n_scores[n] for n in range(start, end+1))
    print(f"  N={start}-{end}: {range_score:.4f} ({100*range_score/sum(per_n_scores.values()):.1f}%)")

Total score: 70.615102

Score breakdown by range:
  N=1-10: 4.3291 (6.1%)
  N=11-50: 14.7036 (20.8%)
  N=51-100: 17.6063 (24.9%)
  N=101-150: 17.1341 (24.3%)
  N=151-200: 16.8421 (23.9%)


In [3]:
# Analyze the theoretical minimum for N=1
# N=1 contributes 0.661 to the score - this is the highest per-N contribution!
# The optimal rotation should minimize the bounding box

import math

def get_tree_bbox_side(angle_deg):
    """Calculate bounding box side for a single tree at given angle."""
    angle_rad = math.radians(angle_deg)
    cos_a, sin_a = math.cos(angle_rad), math.sin(angle_rad)
    
    rotated_x = [cos_a * x - sin_a * y for x, y in zip(TX, TY)]
    rotated_y = [sin_a * x + cos_a * y for x, y in zip(TX, TY)]
    
    width = max(rotated_x) - min(rotated_x)
    height = max(rotated_y) - min(rotated_y)
    return max(width, height)

# Find optimal angle with very fine resolution
best_angle = 0
best_side = float('inf')
for angle in np.arange(0, 360, 0.001):  # 0.001 degree resolution
    side = get_tree_bbox_side(angle)
    if side < best_side:
        best_side = side
        best_angle = angle

print(f"Optimal angle for N=1: {best_angle:.4f}°")
print(f"Optimal side: {best_side:.10f}")
print(f"Optimal score (side^2/1): {best_side**2:.10f}")
print(f"\nBaseline N=1 score: {per_n_scores[1]:.10f}")
print(f"Potential improvement: {per_n_scores[1] - best_side**2:.10f}")

Optimal angle for N=1: 45.0000°
Optimal side: 0.8131727984
Optimal score (side^2/1): 0.6612500000

Baseline N=1 score: 0.6612500000
Potential improvement: -0.0000000000


In [4]:
# Check the baseline N=1 configuration
print("Baseline N=1 configuration:")
print(f"  x={configs[1][0][0]:.10f}")
print(f"  y={configs[1][0][1]:.10f}")
print(f"  angle={configs[1][0][2]:.10f}°")

# Calculate what side the baseline gives
baseline_side = get_tree_bbox_side(configs[1][0][2])
print(f"\nBaseline side: {baseline_side:.10f}")
print(f"Baseline score: {baseline_side**2:.10f}")

Baseline N=1 configuration:
  x=-48.1960861942
  y=58.7709846152
  angle=45.0000000000°

Baseline side: 0.8131727984
Baseline score: 0.6612500000


In [5]:
# Analyze N=2 - can we find a better configuration?
# For N=2, we need to place 2 trees without overlap and minimize bounding box

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polys = [create_tree_polygon(t[0], t[1], t[2]) for t in trees]
    for i in range(len(polys)):
        for j in range(i+1, len(polys)):
            if polys[i].intersects(polys[j]) and not polys[i].touches(polys[j]):
                if polys[i].intersection(polys[j]).area > 1e-15:
                    return True
    return False

print(f"Baseline N=2 score: {per_n_scores[2]:.10f}")
print(f"Baseline N=2 configuration:")
for i, t in enumerate(configs[2]):
    print(f"  Tree {i}: x={t[0]:.6f}, y={t[1]:.6f}, angle={t[2]:.2f}°")

Baseline N=2 score: 0.4507791829
Baseline N=2 configuration:
  Tree 0: x=0.154097, y=-0.038541, angle=203.63°
  Tree 1: x=-0.154097, y=-0.561459, angle=23.63°


In [6]:
# Let's try a different approach: analyze the STRUCTURE of optimal solutions
# What patterns exist in the baseline?

print("Analyzing angle distributions in baseline:")
for n in [5, 10, 20, 50, 100, 200]:
    angles = [t[2] % 360 for t in configs[n]]
    unique_angles = len(set(round(a, 1) for a in angles))
    angle_mean = np.mean(angles)
    angle_std = np.std(angles)
    print(f"N={n}: {unique_angles} unique angles, mean={angle_mean:.1f}°, std={angle_std:.1f}°")

Analyzing angle distributions in baseline:
N=5: 5 unique angles, mean=140.0°, std=97.4°
N=10: 10 unique angles, mean=180.1°, std=125.5°
N=20: 20 unique angles, mean=176.8°, std=101.1°
N=50: 43 unique angles, mean=170.7°, std=106.1°
N=100: 39 unique angles, mean=157.6°, std=90.0°
N=200: 41 unique angles, mean=172.6°, std=91.1°


In [7]:
# Key insight: The baseline is already highly optimized
# The gap of 1.733 points represents improvements that:
# 1. Are NOT in any public source
# 2. Required weeks of C++ optimization
# 3. Accumulated over 900+ submissions

# What CAN we do?
# 1. Try to find ANY per-N improvement, even tiny ones
# 2. Focus on N values where the baseline might be suboptimal

# Let's check which N values have the highest per-N scores (most room for improvement)
per_n_sorted = sorted(per_n_scores.items(), key=lambda x: x[1], reverse=True)
print("Top 20 N values by score (highest = most room for improvement):")
for n, score in per_n_sorted[:20]:
    print(f"  N={n}: {score:.6f}")

Top 20 N values by score (highest = most room for improvement):
  N=1: 0.661250
  N=2: 0.450779
  N=3: 0.434745
  N=5: 0.416850
  N=4: 0.416545
  N=7: 0.399897
  N=6: 0.399610
  N=9: 0.387415
  N=8: 0.385407
  N=15: 0.376949
  N=10: 0.376630
  N=21: 0.376451
  N=20: 0.376057
  N=22: 0.375258
  N=11: 0.374924
  N=16: 0.374128
  N=26: 0.373997
  N=12: 0.372724
  N=13: 0.372294
  N=25: 0.372144


In [8]:
# The theoretical minimum score is related to the packing density
# For identical irregular polygons, the optimal packing density depends on the shape

# Let's calculate the tree area
tree_poly = Polygon(zip(TX, TY))
tree_area = tree_poly.area
print(f"Tree polygon area: {tree_area:.6f}")

# For N trees, the minimum bounding box area is at least N * tree_area / packing_efficiency
# Typical packing efficiency for irregular shapes is 60-80%

print("\nTheoretical lower bounds (assuming 75% packing efficiency):")
for n in [1, 10, 50, 100, 200]:
    min_area = n * tree_area / 0.75
    min_side = np.sqrt(min_area)
    min_score = min_side**2 / n
    actual_score = per_n_scores[n]
    print(f"  N={n}: theoretical min={min_score:.4f}, actual={actual_score:.4f}, gap={actual_score-min_score:.4f}")

Tree polygon area: 0.245625

Theoretical lower bounds (assuming 75% packing efficiency):
  N=1: theoretical min=0.3275, actual=0.6612, gap=0.3337
  N=10: theoretical min=0.3275, actual=0.3766, gap=0.0491
  N=50: theoretical min=0.3275, actual=0.3608, gap=0.0333
  N=100: theoretical min=0.3275, actual=0.3434, gap=0.0159
  N=200: theoretical min=0.3275, actual=0.3375, gap=0.0100


In [9]:
# Summary of findings:
print("="*60)
print("STRATEGIC ANALYSIS SUMMARY")
print("="*60)
print("\n1. BASELINE STATUS:")
print(f"   - Score: 70.615102")
print(f"   - Target: 68.881647")
print(f"   - Gap: 1.733 points (2.5%)")

print("\n2. WHAT WE'VE LEARNED:")
print("   - Baseline is better than ALL public sources")
print("   - Local optimization (perturbation, SA) doesn't work")
print("   - The baseline is at a strong local optimum")

print("\n3. WHAT TOP TEAMS DO (from Eazy Optimizer):")
print("   - C++ with OpenMP parallelization")
print("   - Complex number orbital moves")
print("   - Square pressure gradient descent")
print("   - Multi-scale optimization (1e-3 to 1e-9)")
print("   - Elastic pulse (periodic squeeze/relax)")

print("\n4. POSSIBLE PATHS FORWARD:")
print("   a) Implement gradient-based optimization in Python")
print("   b) Focus on specific N ranges where baseline might be weak")
print("   c) Try constructive heuristics that build from scratch")
print("   d) Implement No-Fit Polygon for efficient collision detection")
print("   e) Use mathematical analysis to find provably optimal small N")

print("\n5. RECOMMENDED NEXT EXPERIMENT:")
print("   Implement gradient-based compaction with square pressure")
print("   This is what the Eazy Optimizer uses and it's pure math, not binary")

STRATEGIC ANALYSIS SUMMARY

1. BASELINE STATUS:
   - Score: 70.615102
   - Target: 68.881647
   - Gap: 1.733 points (2.5%)

2. WHAT WE'VE LEARNED:
   - Baseline is better than ALL public sources
   - Local optimization (perturbation, SA) doesn't work
   - The baseline is at a strong local optimum

3. WHAT TOP TEAMS DO (from Eazy Optimizer):
   - C++ with OpenMP parallelization
   - Complex number orbital moves
   - Square pressure gradient descent
   - Multi-scale optimization (1e-3 to 1e-9)
   - Elastic pulse (periodic squeeze/relax)

4. POSSIBLE PATHS FORWARD:
   a) Implement gradient-based optimization in Python
   b) Focus on specific N ranges where baseline might be weak
   c) Try constructive heuristics that build from scratch
   d) Implement No-Fit Polygon for efficient collision detection
   e) Use mathematical analysis to find provably optimal small N

5. RECOMMENDED NEXT EXPERIMENT:
   Implement gradient-based compaction with square pressure
   This is what the Eazy Optimizer